In [1]:
import os

In [2]:
%pwd

'd:\\Repositories\\ml-churn\\research'

In [3]:
os.chdir("d:/Repositories/ml-churn/")

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name_1: str
    params: dict
    target_column: float


In [15]:
from Churn_analysis.constants import *
from Churn_analysis.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            model_name_1 = config.model_name_1,
            params = params,
            target_column= schema.target_1
            )
        return model_trainer_config
    

In [17]:
import os 
from Churn_analysis import logger
from xgboost import XGBClassifier
import joblib
import pandas as pd


In [22]:
class ModelTrainer():
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def get_model_trainer_object(self):

        """
        This function is responsible for training the model
        """
        train_data_df     = pd.read_csv(self.config.train_data_path)
        
        y_data_train_1 = train_data_df[self.config.target_column]
        X_data_train_1 = train_data_df.drop(columns= self.config.target_column)

        params_xgb  =  self.config.params
    
        xgb_churn = XGBClassifier(**params_xgb,objective='binary:logistic', random_state=42)
        xgb_churn.fit(X_data_train_1,y_data_train_1)


        joblib.dump(xgb_churn, os.path.join(self.config.root_dir,self.config.model_name_1))

        logger.info(f"Model_1 is saved in {os.path.join(self.config.root_dir,self.config.model_name_1)}")



In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.get_model_trainer_object()

except Exception as e:
    raise e

[2023-12-15 16:02:18,372: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-15 16:02:18,374: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 16:02:18,377: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-15 16:02:18,378: INFO: common: created directory at: artifacts]
[2023-12-15 16:02:18,379: INFO: common: created directory at: artifacts/model_trainer]


[16:02:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "reg_lamba" } are not used.



[2023-12-15 16:02:18,679: INFO: 3878663862: Model_1 is saved in artifacts/model_trainer\churn_predictive_model.joblib]
